# Project: Data Warehouse
Samuel Botter Martins

## Teardown AWS resources

This jupyter notebook aims to teardown _all AWS resources_ previously allocated to this project. This includes deleting:
- the IAM role
- the Redshift cluster

Only execute this script after finishing the project!

**NB:** Make sure the IAM ROLE ARN, created in `setup_AWS_resources.ipynb`, is assigned in the `dwh.cfg` file.

## Clean up our AWS resources

**Get the configs**

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# AWS
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')
REGION = config.get('AWS', 'REGION')

# DWH
DWH_CLUSTER_IDENTIFIER = config.get('DWH', 'DWH_CLUSTER_IDENTIFIER')

# IAM
IAM_ROLE_NAME = config.get('IAM', 'ROLE_NAME')
IAM_ROLE_ARN = config.get('IAM', 'ROLE_ARN')

**Delete the Redshift cluster**

In [ ]:
# Create a Reshift python client
import boto3

redshift = boto3.client('redshift',
                        region_name=REGION,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)

In [ ]:
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)

- On the **Redshift dashboard** at AWS, go to _Clusters_ in the side panel.
- Check if the cluster has been removed (see its status)

This may take a while!

**Detach IAM Role and Delete Role**

In [ ]:
# Create a Reshift python client
iam = boto3.client('iam',
                   region_name=REGION,
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

In [ ]:
# detach the assigned policy from the created ROLE
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess')
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn='arn:aws:iam::aws:policy/AmazonRedshiftQueryEditor')

In [ ]:
# delete the role
iam.delete_role(RoleName=IAM_ROLE_NAME)

On **IAM dashboard** on AWS, see if the role has been unassigned from your user. But it keeps active on the **Roles** panel. Delete it manually.